In [ ]:
import sys
import pandas as pd
from pathlib import Path
sys.path.insert(0, '/src')
from shared.generate_note_embeddings import generating_notes, processing_word_vectors
from shared.utils import get_client_class, get_memory_usage
from eliot import start_action, start_task, to_file, log_message
from shared.constants import CLIENT
to_file(sys.stdout)

In [ ]:
clientClass = get_client_class(client=CLIENT)
processed_path = Path('/data/processed/')
raw_path = Path('/data/raw/')

In [ ]:
# Load combined (test, valid, train) datasets so note embeddings can be joined
combined = pd.read_parquet(processed_path/'03-result.parquet')

In [ ]:
# Load Progress note table data
raw_progress_notes = pd.read_parquet(raw_path/'patient_progress_notes.parquet')
raw_progress_notes = raw_progress_notes[~raw_progress_notes['notetext'].isna()]

### ========= Configure the correct note_embeddings_emar_types in client file =========

In [ ]:
# debugging: See which notes to add as eMAR
# [notetype for notetype in raw_progress_notes['progressnotetype'].unique() 
#      if notetype.startswith('e')]

In [ ]:
# debugging: check out one of the note types
# raw_progress_notes[raw_progress_notes.progressnotetype == 'eMar - PRN Medication']

### ============================= END =====================================

In [ ]:
# Fetch filtered Patient Progress Notes
emar_notes, progress_notes = generating_notes(raw_progress_notes, clientClass)
print(emar_notes.shape)
print(progress_notes.shape)

In [ ]:
!  aws s3 cp s3://saiva-models/1/4e78363600a14e65866d8c1ef7ab28fe/artifacts/ /data/models/ --recursive --exclude="*" --include="ft_*"

### Don't forget!! Download ft_emar & ft_non_emar models using linux CLI

<code> aws s3 cp s3://saiva-models/1/4e78363600a14e65866d8c1ef7ab28fe/artifacts/ /data/models/ --recursive --exclude="*" --include="ft_*"


In [ ]:
%%time

final = processing_word_vectors(combined,emar_notes, progress_notes, clientClass)
final.to_parquet(processed_path/'04-result.parquet')

In [ ]:
print(get_memory_usage(combined))
print(get_memory_usage(final))